In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

TATTOO_DATASET_PATH = "/kaggle/input/tattoos/"
COSMETIC_DATASET_PATH = "/kaggle/input/cosmetics-images/"

In [ ]:
def get_images(path):
    return [cv2.imread(os.path.join(path, img)) 
            for img in os.listdir(path)]

def adjust_contrast_brightness(image, contrast, brightness):
    brightness += int(255 * (1 - contrast) / 2)
    return cv2.addWeighted(image, contrast, image, 0, brightness)

def apply_threshold(image):
    blurred = cv2.medianBlur(image, 15)
    greyscaled = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
    thresholded =  cv2.adaptiveThreshold(greyscaled, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY, 11, 2)
    return thresholded

def apply_dilation(image):
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(image, kernel, iterations=1)
    return dilated
    
def get_largest_contours(image, k):
    contours, _ = cv2.findContours(image=image, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
    return contours[1:k+1]

def remove_intersecting_contours(contours):
    for i, a in enumerate(contours):
        print(f"Contour {i}:- ")
        rect1 = cv2.boundingRect(a)
        
        for j, b in enumerate(contours):
            if i == j:
                continue
                
            rect2 = cv2.boundingRect(b)
            if rect1[0] <= rect2[0] and rect1[1] <= rect2[1]:
                print(f"\t{rect1} {rect2}")
                xs = min(rect2[0] + rect2[2], rect1[0] + rect1[2]) - rect2[0]
                ys = min(rect2[1] + rect2[3], rect1[1] + rect1[3]) - rect2[1]
                print(f"\t{xs} {ys}")
                rect_area = rect2[2] * rect2[3]
                overlap_area = xs * ys
                overlap_perc = overlap_area / rect_area
                print(f"\tContour {j}: {overlap_perc}")
            else:
                print(f"\tContour {j}: Skipped")
                
def process_image(image):
    orig = image.copy()
    test = image.copy()
    cont = adjust_contrast_brightness(test, 3, 0)
    thre = apply_threshold(cont)
    dilu = apply_dilation(thre)
    
    return orig, cont, thre, dilu, test
                
def draw_contour_rect(image, contour, circle=False):
    x, y, w, h = cv2.boundingRect(contour)
    image = cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 3)
    
    if circle:
        test_img = cv2.circle(image, (x, y), 8, (255, 0, 0), 2)
        test_img = cv2.circle(image, (x + w, y), 8, (255, 0, 0), 2)
        test_img = cv2.circle(image, (x, y + h), 8, (255, 0, 0), 2)
        test_img = cv2.circle(image, (x + w, y + h), 8, (255, 0, 0), 2)

In [ ]:
images = get_images(COSMETIC_DATASET_PATH)
orig, cont, thre, dilu, test = process_image(images[1])
cnt = get_largest_contours(dilu, 30)

# for i in range(len(cnt)):
#     draw_contour_rect(test, cnt[i])

plt.subplot(2, 2, 1)
plt.imshow(orig)
plt.subplot(2, 2, 2)
plt.imshow(cont)
plt.subplot(2, 2, 3)
plt.imshow(thre)
plt.subplot(2, 2, 4)
plt.imshow(dilu)

In [ ]:
'''
Suggestions:
1. Blur ratio should be ideal enough to separate foreground from its background.
2. Add dilation later
3. Use dialated image to find contours
4. Sort contours w.r.t to the area. Pick top 5 if exists. Tackle overlap case. Eliminate overlapping ones.
5. Return resultant contours. Draw over original image.
6. Use resultant contours to separate foreground from the original image.
'''